In [1]:
import pandas as pd
import glob
import re

In [18]:
ruta_archivo = r'C:\Users\julio\Desktop\The_Bridge\Temario\EDA\Dataset\Banco_de_Espanya\Ocupacion_por_sector\ocup_ramas_clean.xlsx'

# Cargar el archivo en un DataFrame saltando las primeras 5 filas
df = pd.read_excel(ruta_archivo)

df

,DESCRIPCIÓN DE LA SERIE,total_ocupados (miles),i_agricultura,%agr,i_industria,%ind,ii_Industrias extractivas,ii_Industria manufacturera,"iii_Alimentos, bebida y tabaco","iii_Textil, cuero, calzado, madera, papel y artes gráficas","iii_Refino, química, metalurgia, productos de caucho y plástico","iii_Maquinaria, óptica, informática, electrónica y vehículos","ii_Energía electrica, gas, vapor, aire acondicionado, agua, saneamientos y residuos",i_Construcción,%cons,i_Servicios,%serv
0,MAR 2002,16482.3,-,NaN,-,NaN,-,-,-,-,-,-,-,-,NaN,-,NaN
1,JUN 2002,16766.9,-,NaN,-,NaN,-,-,-,-,-,-,-,-,NaN,-,NaN
2,SEP 2002,16919.3,-,NaN,-,NaN,-,-,-,-,-,-,-,-,NaN,-,NaN
3,DIC 2002,16991.9,-,NaN,-,NaN,-,-,-,-,-,-,-,-,NaN,-,NaN
4,MAR 2003,17092.7,-,NaN,-,NaN,-,-,-,-,-,-,-,-,NaN,-,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,DIC 2021,20184.9,840.1,0.041620,2765.2,0.136993,30.4,2489.3,517.3,346.6,542.6,1082.7,245.4,1285.9,0.063706,15293.7,0.757680
80,MAR 2022,20084.7,827.7,0.041210,2697.1,0.134286,27.8,2423.7,506.9,344.3,527.6,1045,245.6,1316.3,0.065537,15243.6,0.758966
81,JUN 2022,20468.0,789.3,0.038563,2776.6,0.135656,29.1,2503.6,534.9,359.5,556.5,1052.9,243.9,1338.2,0.065380,15563.9,0.760402
82,SEP 2022,20545.7,729,0.035482,2809.8,0.136759,29.7,2548.2,536.6,352.2,561.8,1097.7,231.8,1328.8,0.064675,15678.2,0.763089


In [19]:
# Diccionario para mapear las abreviaturas de los meses a sus equivalentes en inglés
meses = {
    'ENE': 'Jan',
    'FEB': 'Feb',
    'MAR': 'Mar',
    'ABR': 'Apr',
    'MAY': 'May',
    'JUN': 'Jun',
    'JUL': 'Jul',
    'AGO': 'Aug',
    'SEP': 'Sep',
    'OCT': 'Oct',
    'NOV': 'Nov',
    'DIC': 'Dec'
}

# Reemplazar las abreviaturas de los meses con sus equivalentes en inglés
for mes_abrev, mes_ingles in meses.items():
    df['DESCRIPCIÓN DE LA SERIE'] = df['DESCRIPCIÓN DE LA SERIE'].str.replace(mes_abrev, mes_ingles)

# Convertir a formato de fecha
df['DESCRIPCIÓN DE LA SERIE'] = pd.to_datetime(df['DESCRIPCIÓN DE LA SERIE'], format='%b %Y')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 17 columns):
 #   Column                                                                               Non-Null Count  Dtype         
---  ------                                                                               --------------  -----         
 0   DESCRIPCIÓN DE LA SERIE                                                              84 non-null     datetime64[ns]
 1   total_ocupados (miles)                                                               84 non-null     float64       
 2   i_agricultura                                                                        84 non-null     object        
 3   %agr                                                                                 60 non-null     float64       
 4   i_industria                                                                          84 non-null     object        
 5   %ind                                         

In [20]:
# DF CON LOS OCUPADOS TRIMESTRALMENTE
df_total_ocupados = df[['DESCRIPCIÓN DE LA SERIE', 'total_ocupados (miles)']]
df_total_ocupados

,DESCRIPCIÓN DE LA SERIE,total_ocupados (miles)
0,2002-03-01,16482.3
1,2002-06-01,16766.9
2,2002-09-01,16919.3
3,2002-12-01,16991.9
4,2003-03-01,17092.7
...,...,...
79,2021-12-01,20184.9
80,2022-03-01,20084.7
81,2022-06-01,20468.0
82,2022-09-01,20545.7


In [21]:
# Calcular el promedio trimestral de total_ocupados
promedio_trimestral = df_total_ocupados.resample('Q', on='DESCRIPCIÓN DE LA SERIE').mean()

# Agrupar por año y obtener los valores promedio anuales de ocupados
promedio_anual = promedio_trimestral.resample('Y').mean()

# Redefinir el índice del nuevo DataFrame
promedio_anual.reset_index(inplace=True)

promedio_anual['year'] = promedio_anual['DESCRIPCIÓN DE LA SERIE'].dt.year

# Cambiar el nombre de la columna 'total_ocupados' a 'promedio_ocupados'
promedio_anual.rename(columns={'total_ocupados (miles)': 'promedio_ocupados'}, inplace=True)

# Eliminar la columna 'DESCRIPCIÓN DE LA SERIE' si no se necesita más
promedio_anual.drop(columns='DESCRIPCIÓN DE LA SERIE', inplace=True)

promedio_anual

,promedio_ocupados,year
0,16790.100,2002
1,17475.600,2003
2,18142.250,2004
3,19207.000,2005
4,19939.100,2006
5,20579.925,2007
6,20469.650,2008
7,19106.850,2009
8,18724.475,2010
9,18421.425,2011


In [23]:
# DF CON LOS OCUPADOS TRIMESTRALMENTE POR SECTOR

df_sector_tri = df.iloc[:, [0, 2, 3, 4, 5, 13, 14, 15, 16]]
df_sector_tri = df_sector_tri.dropna()

df_sector_tri

,DESCRIPCIÓN DE LA SERIE,i_agricultura,%agr,i_industria,%ind,i_Construcción,%cons,i_Servicios,%serv
24,2008-03-01,870.1,0.042197,3352.8,0.162599,2679.5,0.129947,13717.6,0.665257
25,2008-06-01,830.6,0.040229,3284.3,0.159070,2559.4,0.123960,13972.6,0.676741
26,2008-09-01,798.8,0.038859,3231.1,0.157182,2418,0.117628,14108.6,0.686336
27,2008-12-01,813.2,0.040548,3078.5,0.153501,2182.6,0.108829,13981,0.697122
28,2009-03-01,842.5,0.043688,2930.6,0.151967,1977.6,0.102549,13533.8,0.701800
29,2009-06-01,790.8,0.041286,2829.1,0.147701,1927.8,0.100646,13606.6,0.710372
30,2009-09-01,740.6,0.038778,2752,0.144096,1850.7,0.096903,13755.1,0.720223
31,2009-12-01,778.6,0.041217,2718.6,0.143914,1803.1,0.095451,13590.2,0.719424
32,2010-03-01,827.7,0.044374,2637.9,0.141420,1668.2,0.089434,13519.1,0.724772
33,2010-06-01,772.8,0.041214,2656.7,0.141682,1698.1,0.090560,13623.5,0.726544


In [24]:
# Calcular el promedio trimestral de total_ocupados
sector_trimestral = df_sector_tri.resample('Q', on='DESCRIPCIÓN DE LA SERIE').mean()

# Agrupar por año y obtener los valores promedio anuales de ocupados
sector_anual = sector_trimestral.resample('Y').mean()

# Redefinir el índice del nuevo DataFrame
sector_anual.reset_index(inplace=True)

sector_anual['year'] = sector_anual['DESCRIPCIÓN DE LA SERIE'].dt.year

# Cambiar el nombre de la columna 'total_ocupados' a 'promedio_ocupados'
sector_anual.rename(columns={'total_ocupados (miles)': 'promedio_ocupados'}, inplace=True)

# Eliminar la columna 'DESCRIPCIÓN DE LA SERIE' si no se necesita más
sector_anual.drop(columns='DESCRIPCIÓN DE LA SERIE', inplace=True)

sector_anual

,i_agricultura,%agr,i_industria,%ind,i_Construcción,%cons,i_Servicios,%serv,year
0,828.175,0.040458,3236.675,0.158088,2459.875,0.120091,13944.95,0.681364,2008
1,788.125,0.041242,2807.575,0.146920,1889.8,0.098887,13621.425,0.712955,2009
2,786.075,0.041987,2650.45,0.141552,1651.35,0.088189,13636.6,0.728272,2010
3,755.25,0.041017,2604.7,0.141399,1403.9,0.076189,13657.575,0.741395,2011
4,743.375,0.042171,2483.7,0.140857,1161.325,0.065843,13244.2,0.751124,2012
5,736.575,0.042978,2355.525,0.137442,1029.475,0.060070,13017.475,0.759512,2013
6,735.85,0.042470,2379.95,0.137202,993.5,0.057265,13234.85,0.763063,2014
7,736.75,0.041236,2482.325,0.138953,1073.65,0.060104,13573.3,0.759706,2015
8,774.525,0.042232,2522.225,0.137512,1073.85,0.058539,13970.95,0.761716,2016
9,819.5,0.043555,2647.375,0.140623,1128.325,0.059929,14229.6,0.755894,2017
